### ***Scroll down for part 3***

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

## Part 1
1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, <https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M>, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

3. To create the above dataframe:
 * The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
 * Only process the cells that have an assigned borough. Ignore cells with a borough that is **Not assigned**.
 * More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that **M5A** is listed twice and has two neighborhoods: **Harbourfront** and **Regent Park**. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in **row 11** in the above table. (***Note:*** This problem seem to have been already corrected in the Wiki data.)
 * If a cell has a borough but a **Not assigned** neighborhood, then the neighborhood will be the same as the borough. (***Note:*** This problem seems to have been already corrected in the Wiki data.)
 * Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
 * In the last cell of your notebook, use the **.shape** method to print the number of rows of your dataframe.
 
4. Submit a link to your Notebook on your Github repository. (**10 marks**)

**Note**: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use *pandas* to read the table into a *pandas* dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: <http://beautiful-soup-4.readthedocs.io/en/latest/>

Use *pandas*, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

#### Import packages

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

#### Scrape the list of Toronto's postal codes

In [2]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text
soup = BeautifulSoup(source, 'xml')
table = soup.find('table')

#### Convert table to df with postal code, borough, and neighbourhood info

In [3]:
def TableRowsToDataRows(row):
    '''Convert soup table row to list of data'''
    # Strip html and whitespace from each cell of row.
    return list(map(lambda cell: cell.get_text().strip(), row.find_all()))

# Get list of all rows with formated data and put into dataframe. First row of data is column names.
data = list(map(TableRowsToDataRows, table.find_all('tr')))
df = pd.DataFrame(data[1:], columns = data[0])
df.head(12)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


#### Clean dataframe
Remove rows where Borough is 'Not assigned'. 
There are no postal codes were Neighbourhood is subsequently not assigned. Postal Codes with multiple neighbourhoods have already been grouped. 

In [4]:
df=df[df['Borough']!='Not assigned']
df[df['Neighbourhood']=='Not assigned']

,Postal Code,Borough,Neighbourhood


#### Review dataframe
Reset index and view dataframe. Examine it's shape.

In [5]:
df.reset_index(drop=True, inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
df.shape

(103, 3)

## Part 2
Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

##### Note:
In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. 

#### ***Note:***
Using `geocode` with Google's API is no longer free. Based on the suggestions from others, I've instead implemented this using `arcgis` instead of `google`. 

In [7]:
# Run this code if geocoder is not installed.
#!conda install -c conda-forge geocoder --yes
import geocoder # import geocoder

Define a function that uses `geocode` to fetch latitude and longitude for the postal code from a row of the dataframe.

In [8]:
def get_geocode_for_row(row):
    '''
    Fetches coordinates for a postal code given a row in the dataframe.
    Returns a list of the form [latititude, longitude].
    '''
    print('\rFetching data for postal code: ' + row['Postal Code'], end='')
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(row['Postal Code']))
    return [round(coord,5) for coord in g.geojson['features'][0]['geometry']['coordinates']][::-1]

Apply this to each row of the dataframe and add columns for 'Latitude' and 'Longitude'. ***Worning:*** This takes a long time for some reason. I've written in the code a way to track the progress by displaying the postal code that is currently being checked.

In [9]:
df['Latitude'], df['Longitude'] = zip(*df.apply(get_geocode_for_row, axis=1))
print('\rDone!',end='                              ')

Done!                              

In [10]:
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75188,-79.33036
1,M4A,North York,Victoria Village,43.73042,-79.31282
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72321,-79.45141
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302


##### Alternate method
An alternate method (since the geocoder method may be unreliable) is to simply fetch the geospatial data that has already been fetched by the course coordinaters. I've commented this method out here.

***Note:*** The geospatial coordinates from `arcgis` are slightly different from those provided by Google. 

In [11]:
#geo_df=pd.read_csv('http://cocl.us/Geospatial_data')
#geo_df.rename(columns={'Postalcode':'Postal Code'}, inplace=True)
#geo_df.head()
#df_merge = pd.merge(df, geo_df, on='Postal Code')
#df_merge.head()

## Part 3
Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

#### Extract only neighbourhoods in boroughs with "Toronto" in the name.

In [12]:
toronto_data=df[df['Borough'].str.contains("Toronto")]
toronto_data.reset_index(drop=True, inplace=True)
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65736,-79.37818
3,M5C,Downtown Toronto,St. James Town,43.65143,-79.37557
4,M4E,East Toronto,The Beaches,43.67703,-79.29542


#### Import packages
 * Import `Nominatim` from `geopy.geocoders` to convert address to latitude and longitude. 
 * Import `folium` to render maps.
 * Import `json_normalize` from `pandas.io.json` to render maps to transform JSON files into pandas dataframe.

In [13]:
#Uncomment this line if conda is not installed.
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#Uncomment this line if folium is not installed.
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#### Set up
Get geographical coordinates of Toronto using `Nominatim`.

In [14]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [15]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [16]:
CLIENT_ID = '3UWHZSUQXDPJOBOFOXUGPLXIFT5LR4NYHSPATFAJQ3OGUQBR' # your Foursquare ID
CLIENT_SECRET = 'IEJUE22JXGU255UQKYUQBEFFF5R4XPUAH0M0UOV1GVMG0DYE' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3UWHZSUQXDPJOBOFOXUGPLXIFT5LR4NYHSPATFAJQ3OGUQBR
CLIENT_SECRET:IEJUE22JXGU255UQKYUQBEFFF5R4XPUAH0M0UOV1GVMG0DYE


#### Explore the first neighborhood in our dataframe.

In [17]:
toronto_data.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

Get the neighborhood's latitude and longitude values.

In [18]:
neighbourhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.65514, -79.36265.


#### Get the top 100 venues that are in this neighbourhood within a radius of 500 meters.
Create the GET request URL. Name your URL **url**.

In [19]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=3UWHZSUQXDPJOBOFOXUGPLXIFT5LR4NYHSPATFAJQ3OGUQBR&client_secret=IEJUE22JXGU255UQKYUQBEFFF5R4XPUAH0M0UOV1GVMG0DYE&v=20180604&ll=43.65514,-79.36265&radius=500&limit=100'

Send the GET request and examine the resutls

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f2981e92378a6144bd1fb0b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 22,
  'suggestedBounds': {'ne': {'lat': 43.659640004500005,
    'lng': -79.35644190916516},
   'sw': {'lat': 43.6506399955, 'lng': -79.36885809083485}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53b8466a498e83df908c3f21',
       'name': 'Tandem Coffee',
       'location': {'address': '368 King St E',
        'crossStreet': 'at Trinity St',
        'lat': 43.65355870959944,
        'lng': -79.36180945913513,
        'labeledLatLngs': [{'label': 'd

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
3,The Yoga Lounge,Yoga Studio,43.655515,-79.364955
4,Body Blitz Spa East,Spa,43.654735,-79.359874


How many venues were returned by Foursquare?

In [23]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

22 venues were returned by Foursquare.


#### Create a function that will process a neighbourhood and find nearby businesses.

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [26]:
print(toronto_venues.shape)
toronto_venues.head()

(1747, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65514,-79.36265,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65514,-79.36265,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65514,-79.36265,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65514,-79.36265,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.65514,-79.36265,Body Blitz Spa East,43.654735,-79.359874,Spa


In [27]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,67,67,67,67,67,67
"Brockton, Parkdale Village, Exhibition Place",84,84,84,84,84,84
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",68,68,68,68,68,68
Central Bay Street,74,74,74,74,74,74
Christie,10,10,10,10,10,10
Church and Wellesley,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,25,25,25,25,25,25


In [28]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 222 uniques categories.


### Analyze neighbourhoods

In [29]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
manhattan_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Neighbourhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront"
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront"
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront"
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,"Regent Park, Harbourfront"
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront"


In [30]:
toronto_onehot.shape

(1747, 223)

In [31]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.00,0.000000,0.014925,0.000000,0.000000,0.000000,...,0.00,0.000000,0.014925,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.014925
1,"Brockton, Parkdale Village, Exhibition Place",0.011905,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.023810,0.011905,...,0.00,0.000000,0.011905,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.011905
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.020000,0.00,0.000000,0.010000,0.000000,0.010000,0.030000,...,0.00,0.000000,0.020000,0.000000,0.000000,0.010000,0.00,0.000000,0.00,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.014706,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.014706,0.00,0.014706
4,Central Bay Street,0.000000,0.000000,0.000000,0.00,0.000000,0.013514,0.013514,0.000000,0.000000,...,0.00,0.000000,0.000000,0.013514,0.013514,0.013514,0.00,0.000000,0.00,0.000000
5,Christie,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000
6,Church and Wellesley,0.000000,0.012821,0.012821,0.00,0.000000,0.000000,0.000000,0.012821,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.012821
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.040000,0.00,0.000000,0.010000,0.000000,0.000000,0.010000,...,0.00,0.000000,0.010000,0.000000,0.000000,0.010000,0.00,0.000000,0.00,0.010000
8,Davisville,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000
9,Davisville North,0.000000,0.000000,0.142857,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000


In [32]:
toronto_grouped.shape

(39, 223)

In [33]:
num_top_venues = 5

for neighbourhood in toronto_grouped['Neighbourhood']:
    print("----"+neighbourhood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == neighbourhood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1          Restaurant  0.04
2              Bakery  0.03
3                Café  0.03
4  Seafood Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.07
1     Coffee Shop  0.06
2      Restaurant  0.05
3             Bar  0.05
4  Sandwich Place  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0         Coffee Shop  0.10
1               Hotel  0.05
2  Italian Restaurant  0.03
3          Restaurant  0.03
4    Asian Restaurant  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Coffee Shop  0.09
1  Italian Restaurant  0.07
2                 Bar  0.04
3                Café  0.04
4   Electronics Store  0.03


----Central Bay Street----
                       venue  freq
0

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Bakery,Italian Restaurant,Hotel,Seafood Restaurant,Café,Beer Bar,Cheese Shop,Farmers Market
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Restaurant,Bar,Gift Shop,Sandwich Place,Nightclub,Supermarket,Italian Restaurant,Furniture / Home Store
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Hotel,Café,Italian Restaurant,Restaurant,Asian Restaurant,Seafood Restaurant,Burrito Place,Steakhouse,Concert Hall
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Italian Restaurant,Bar,Café,Electronics Store,Bakery,French Restaurant,Restaurant,Gym / Fitness Center,Park
4,Central Bay Street,Coffee Shop,Clothing Store,Plaza,Italian Restaurant,Electronics Store,Sushi Restaurant,Bubble Tea Shop,Hotel,Middle Eastern Restaurant,Cosmetics Shop


### Cluster neighbourhoods

In [36]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [37]:
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighbourhoods_venues_sorted


,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Berczy Park,Coffee Shop,Restaurant,Bakery,Italian Restaurant,Hotel,Seafood Restaurant,Café,Beer Bar,Cheese Shop,Farmers Market
1,0,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Restaurant,Bar,Gift Shop,Sandwich Place,Nightclub,Supermarket,Italian Restaurant,Furniture / Home Store
2,0,"Business reply mail Processing Centre, South C...",Coffee Shop,Hotel,Café,Italian Restaurant,Restaurant,Asian Restaurant,Seafood Restaurant,Burrito Place,Steakhouse,Concert Hall
3,0,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Italian Restaurant,Bar,Café,Electronics Store,Bakery,French Restaurant,Restaurant,Gym / Fitness Center,Park
4,0,Central Bay Street,Coffee Shop,Clothing Store,Plaza,Italian Restaurant,Electronics Store,Sushi Restaurant,Bubble Tea Shop,Hotel,Middle Eastern Restaurant,Cosmetics Shop
5,0,Christie,Café,Grocery Store,Coffee Shop,Italian Restaurant,Baby Store,Candy Store,Playground,Farmers Market,Dumpling Restaurant,Farm
6,0,Church and Wellesley,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Pizza Place,Pub,Hotel,Dance Studio,Café
7,0,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Hotel,Café,Gym,American Restaurant,Italian Restaurant,Japanese Restaurant,Beer Bar,Seafood Restaurant
8,0,Davisville,Dessert Shop,Sandwich Place,Italian Restaurant,Café,Pizza Place,Coffee Shop,Restaurant,Sushi Restaurant,Gas Station,Thai Restaurant
9,0,Davisville North,Park,Food & Drink Shop,American Restaurant,Hotel,Breakfast Spot,Gym / Fitness Center,Department Store,Farmers Market,Ethiopian Restaurant,Event Space


In [38]:
# add clustering labels
#neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.dropna(inplace=True)
toronto_merged # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265,0,Coffee Shop,Breakfast Spot,Restaurant,Thai Restaurant,Health Food Store,Pub,Distribution Center,Electronics Store,Event Space,Spa
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302,0,Coffee Shop,Café,Park,Sandwich Place,Pub,Music Store,Office,Burger Joint,Chinese Restaurant,Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65736,-79.37818,0,Coffee Shop,Clothing Store,Hotel,Café,Cosmetics Shop,Italian Restaurant,Japanese Restaurant,Theater,Bubble Tea Shop,Fast Food Restaurant
3,M5C,Downtown Toronto,St. James Town,43.65143,-79.37557,0,Seafood Restaurant,Café,Restaurant,Coffee Shop,Clothing Store,Cocktail Bar,American Restaurant,Cosmetics Shop,Gastropub,Italian Restaurant
4,M4E,East Toronto,The Beaches,43.67703,-79.29542,5,Health Food Store,Pub,Trail,Neighborhood,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
5,M5E,Downtown Toronto,Berczy Park,43.64531,-79.37368,0,Coffee Shop,Restaurant,Bakery,Italian Restaurant,Hotel,Seafood Restaurant,Café,Beer Bar,Cheese Shop,Farmers Market
6,M5G,Downtown Toronto,Central Bay Street,43.65609,-79.38493,0,Coffee Shop,Clothing Store,Plaza,Italian Restaurant,Electronics Store,Sushi Restaurant,Bubble Tea Shop,Hotel,Middle Eastern Restaurant,Cosmetics Shop
7,M6G,Downtown Toronto,Christie,43.66878,-79.42071,0,Café,Grocery Store,Coffee Shop,Italian Restaurant,Baby Store,Candy Store,Playground,Farmers Market,Dumpling Restaurant,Farm
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.64970,-79.38258,0,Coffee Shop,Café,Gym,Hotel,Japanese Restaurant,Restaurant,American Restaurant,Steakhouse,Salad Place,Asian Restaurant
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.66509,-79.43871,0,Park,Café,Smoke Shop,Pharmacy,Bus Line,Gym,Art Gallery,Grocery Store,Middle Eastern Restaurant,Furniture / Home Store


In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",0,Coffee Shop,Breakfast Spot,Restaurant,Thai Restaurant,Health Food Store,Pub,Distribution Center,Electronics Store,Event Space,Spa
1,"Queen's Park, Ontario Provincial Government",0,Coffee Shop,Café,Park,Sandwich Place,Pub,Music Store,Office,Burger Joint,Chinese Restaurant,Restaurant
2,"Garden District, Ryerson",0,Coffee Shop,Clothing Store,Hotel,Café,Cosmetics Shop,Italian Restaurant,Japanese Restaurant,Theater,Bubble Tea Shop,Fast Food Restaurant
3,St. James Town,0,Seafood Restaurant,Café,Restaurant,Coffee Shop,Clothing Store,Cocktail Bar,American Restaurant,Cosmetics Shop,Gastropub,Italian Restaurant
5,Berczy Park,0,Coffee Shop,Restaurant,Bakery,Italian Restaurant,Hotel,Seafood Restaurant,Café,Beer Bar,Cheese Shop,Farmers Market
6,Central Bay Street,0,Coffee Shop,Clothing Store,Plaza,Italian Restaurant,Electronics Store,Sushi Restaurant,Bubble Tea Shop,Hotel,Middle Eastern Restaurant,Cosmetics Shop
7,Christie,0,Café,Grocery Store,Coffee Shop,Italian Restaurant,Baby Store,Candy Store,Playground,Farmers Market,Dumpling Restaurant,Farm
8,"Richmond, Adelaide, King",0,Coffee Shop,Café,Gym,Hotel,Japanese Restaurant,Restaurant,American Restaurant,Steakhouse,Salad Place,Asian Restaurant
9,"Dufferin, Dovercourt Village",0,Park,Café,Smoke Shop,Pharmacy,Bus Line,Gym,Art Gallery,Grocery Store,Middle Eastern Restaurant,Furniture / Home Store
10,"Harbourfront East, Union Station, Toronto Islands",0,Coffee Shop,Hotel,Japanese Restaurant,Plaza,Park,Boat or Ferry,Deli / Bodega,Lounge,Sandwich Place,Bubble Tea Shop
